In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ERA_V2/Assignment21/

/content/drive/MyDrive/ERA_V2/Assignment21


In [ ]:
!pip install tiktoken
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.3 MB/s

--------

In [ ]:
!python train_get2-9-speedup9.py

using device: cuda
loaded 338025 tokens
1 epoch = 20 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
/content/drive/MyDrive/ERA_V2/Assignment21/train_get2-9-speedup9.py:306: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
step0 | loss: 10.9752197265625 | dt: 26205.27ms | tok/sec:  625.22 | norm: 7.11
step1 | loss: 9.704957962036133 | dt: 471.73ms | tok/sec:  34731.84 | norm: 5.35
step2 | loss: 9.325849533081055 | dt: 472.31ms | tok/sec:  34689.16 | norm: 6.38
step3 | loss: 9.35608959197998 | dt: 484.72ms | tok/sec:  33800.91 | norm: 4.46
step4 | loss: 8.888101577758789 | dt: 481.43ms | tok/sec:  34031.79 | norm: 3.77
step5 | loss: 8.705806732177734 | dt: 488.57ms | tok/sec:  33534.60 | norm: 2.09
step6 | loss: 8.569231986999512 | dt: 482.29ms | tok/sec:  3397

## Load the saved model

--------

In [ ]:
#import model and the configuration
import torch
import torch.nn.functional as F
from model_gpt2 import GPT, GPTConfig
#set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'





In [ ]:
checkpoint = torch.load('model.pth', map_location=device)
gptconf = GPTConfig()
model = GPT(gptconf)
state_dict = checkpoint
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

model.eval()
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)

## prediction

In [ ]:
num_return_sequences = 5
max_length = 30

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype= torch.long) # (8,) #check tiktoken app
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
x = tokens.to(device)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x)[0] # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)
decoded_texts = []
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)

    # Store the decoded text in the list
    #decoded_texts.append(decoded)
    decoded_texts.append("\n" + decoded + "\n")

  # Join all the decoded texts into a single string and print it
final_decoded_text = "\n".join(decoded_texts)
print(final_decoded_text)
'''
# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)
'''


Hello, I'm a language model,

Morerish; being
Her rightful smallest Northines unhe,
'er owe with their pains


Hello, I'm a language model,

More than to heard the netoured upon
In sign'd; my rest.
On the drunk


Hello, I'm a language model, being being
To hear,
More than in Margaret. I tenderly fond horn.NORI t


Hello, I'm a language model,

More affrancho myessor of Richard's Few thousand truth and my glollo too e.



Hello, I'm a language model,
'er king,
 I''ld fool.
 my defiance, the kodp of ourselves



'\n# print the generated text\nfor i in range(num_return_sequences):\n    tokens = x[i, :max_length].tolist()\n    decoded = enc.decode(tokens)\n    print(">", decoded)\n'

## Gradio App

In [ ]:
import gradio as gr
import tiktoken

In [ ]:
# gradio function
decoded_texts = []
def generate_output(max_length, num_return_sequences):
  enc = tiktoken.get_encoding('gpt2')
  tokens = enc.encode("Hello, I'm a language model,")
  tokens = torch.tensor(tokens, dtype= torch.long) # (8,) #check tiktoken app
  tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
  x = tokens.to(device)

  torch.manual_seed(42)
  torch.cuda.manual_seed(42)
  while x.size(1) < max_length:
      # forward the model to get the logits
      with torch.no_grad():
          logits = model(x)[0] # (B, T, vocab_size)
          # take the logits at the last position
          logits = logits[:, -1, :] # (B, vocab_size)
          # get the probabilities
          probs = F.softmax(logits, dim=-1)
          # do top-k sampling of 50 (huggingface pipeline default)
          # topk_probs here becomes (5, 50), topk_indices is (5, 50)
          topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
          # select a token from the top-k probabilities
          # note: multinomial does not demand the input to sum to 1
          ix = torch.multinomial(topk_probs, 1) # (B, 1)
          # gather the corresponding indices
          xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
          # append to the sequence
          x = torch.cat((x, xcol), dim=1)
  decoded_texts = []
  for i in range(num_return_sequences):
      tokens = x[i, :max_length].tolist()
      decoded = enc.decode(tokens)

      # Store the decoded text in the list
      #decoded_texts.append(decoded)
      #decoded_texts.append(f" {decoded}")
      decoded_texts.append("\n" + decoded + "\n")


    # Join all the decoded texts into a single string and print it
  final_decoded_text = "".join(decoded_texts)
  #print(final_decoded_text)
  return final_decoded_text


# instance gradio applications
title = "GPT2-Shakespeare Like Text Generation"
description = "GPT2-Model trained to generates text in the style of William Shakespeare."

demo = gr.Interface(
fn = generate_output,
    inputs = [
        gr.Number(value = 50,label = "Length of Sentence",info = "Length of the sample sentence you wish to generate."),
        gr.Number(value = 2,label = "Sequence",info = "How many sequence you wish to generate.")
    ],
    outputs = [gr.TextArea(lines = 5,label="Sequence Output")],
    title = title,
    description = description
)

# launch interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://63dbe028af862991b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
